Purpose: take Kuprov algorithm (which is much slower than mine) and try to make it faster than mine with cython.

In [3]:
%load_ext cython
import Cython
print(Cython.__version__)

import numpy as np


0.28.5


In [4]:
from simulation_data import spin8

Testing indicated, surprisingly, that Kuprov algorithm ~~had almost identical speeds with lil_matrix, csr_matrix, *and* regular numpy dense matrices.~~ Update: actually, switching to np.kron made the dense version ca 30x *faster* somehow! So, won't worry with cython and scipy integration, just cython and numpy.

Kuprov version of hamiltonian:

In [5]:
def kuprov_H(v, J):
    sigma_x = np.matrix([[0, 1 / 2], [1 / 2, 0]])
    sigma_y = np.matrix([[0, -1j / 2], [1j / 2, 0]])
    sigma_z = np.matrix([[1 / 2, 0], [0, -1 / 2]])
    unit = np.matrix([[1, 0], [0, 1]])

    nspins = len(v)
    Lx = []
    Ly = []
    Lz = []

    for n in range(nspins):
        Lx_current = 1
        Ly_current = 1
        Lz_current = 1

        for k in range(nspins):
            if k == n:
                Lx_current = np.kron(Lx_current, sigma_x)
                Ly_current = np.kron(Ly_current, sigma_y)
                Lz_current = np.kron(Lz_current, sigma_z)
            else:
                Lx_current = np.kron(Lx_current, unit)
                Ly_current = np.kron(Ly_current, unit)
                Lz_current = np.kron(Lz_current, unit)

        Lx.append(Lx_current)
        Ly.append(Ly_current)
        Lz.append(Lz_current)

    H = np.zeros((2**nspins, 2**nspins), dtype=np.complex128)
    for n in range(nspins):
        H += v[n] * Lz[n]

    for n in range(nspins):
        for k in range(nspins):
            if n != k:
                H += 0.5 * J[n, k] * (Lx[n]*Lx[k] + Ly[n]*Ly[k] + Lz[n]*Lz[k])

    return H

In [6]:
dense_result = %timeit -o kuprov_H(*spin8())
dense_result.average


287 ms ± 15.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


0.28665980928573326

In [7]:
DTYPE = np.complex128

Let's try dividing the function into smaller functions. Also, let's try defining a single matrix for stacked Lx, Ly, and Lz.

In [21]:
def spin_operators(nspins):
    sigma_x = np.array([[0, 1 / 2], [1 / 2, 0]])
    sigma_y = np.array([[0, -1j / 2], [1j / 2, 0]])
    sigma_z = np.array([[1 / 2, 0], [0, -1 / 2]])
    unit = np.array([[1, 0], [0, 1]])

#     Lx = []
#     Ly = []
#     Lz = []
    L = np.empty((3, nspins, 2**nspins, 2**nspins), dtype=DTYPE)
    print(L.shape)
    for n in range(nspins):
        Lx_current = 1
        Ly_current = 1
        Lz_current = 1

        for k in range(nspins):
            if k == n:
                Lx_current = np.kron(Lx_current, sigma_x)
                Ly_current = np.kron(Ly_current, sigma_y)
                Lz_current = np.kron(Lz_current, sigma_z)
            else:
                Lx_current = np.kron(Lx_current, unit)
                Ly_current = np.kron(Ly_current, unit)
                Lz_current = np.kron(Lz_current, unit)

#         Lx.append(Lx_current)
#         Ly.append(Ly_current)
#         Lz.append(Lz_current)
        L[0][n] = Lx_current
        L[1][n] = Ly_current
        L[2][n] = Lz_current
        
    return L

In [24]:
L = spin_operators(2)
L[1]

(3, 2, 4, 4)


array([[[0.+0.j , 0.+0.j , 0.-0.5j, 0.+0.j ],
        [0.+0.j , 0.+0.j , 0.+0.j , 0.-0.5j],
        [0.+0.5j, 0.+0.j , 0.+0.j , 0.+0.j ],
        [0.+0.j , 0.+0.5j, 0.+0.j , 0.+0.j ]],

       [[0.+0.j , 0.-0.5j, 0.+0.j , 0.-0.j ],
        [0.+0.5j, 0.+0.j , 0.+0.j , 0.+0.j ],
        [0.+0.j , 0.-0.j , 0.+0.j , 0.-0.5j],
        [0.+0.j , 0.+0.j , 0.+0.5j, 0.+0.j ]]])

In [44]:
def hamiltonian(v, J, L):
#     H = np.zeros((2**nspins, 2**nspins), dtype=np.complex128)
#     for n in range(nspins):
#         H += v[n] * Lz[n]
    nspins = len(v)
    print(L[0][0])
    print(L[0][1])
    print(L[0][0]*L[0][1])
    H = np.tensordot(v, L[2], axes=1)

    for n in range(nspins):
        for k in range(nspins):
            if n != k:
                H += 0.5 * J[n, k] * (L[0][n]*L[0][k] + L[1][n]*L[1][k] + L[2][n]*L[2][k])

    return H
    

In [45]:
m1 = np.array([[0, 1],[2, 3]])
m2 = np.array([[10, 20], [30, 40]])
a = np.array([m1, m2])
v = np.array([2, 4])
np.tensordot(v, a, axes=1)

array([[ 40,  82],
       [124, 166]])

In [46]:
v, J = spin8()
L = spin_operators(8)
H1 = kuprov_H(v, J)
H2 = hamiltonian(v, J, L)
print(H1)
print(H2)


(3, 8, 256, 256)
[[0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 ...
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]]
[[0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 ...
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]]
[[0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 ...
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]]
[[ 741.25+0.j    0.  +0.j    0.  +0.j ...    0.  +0.j    0.  +0.j
     0.  +0.j]
 [

In [18]:
%%cython -a
import numpy as np

def c1(v, J):
    sigma_x = np.matrix([[0, 1 / 2], [1 / 2, 0]])
    sigma_y = np.matrix([[0, -1j / 2], [1j / 2, 0]])
    sigma_z = np.matrix([[1 / 2, 0], [0, -1 / 2]])
    unit = np.matrix([[1, 0], [0, 1]])

    cdef Py_ssize_t nspins = len(v)
    Lx = []
    Ly = []
    Lz = []
    
    cdef Py_ssize_t n, k

    for n in range(nspins):
        Lx_current = 1
        Ly_current = 1
        Lz_current = 1

        for k in range(nspins):
            if k == n:
                Lx_current = np.kron(Lx_current, sigma_x)
                Ly_current = np.kron(Ly_current, sigma_y)
                Lz_current = np.kron(Lz_current, sigma_z)
            else:
                Lx_current = np.kron(Lx_current, unit)
                Ly_current = np.kron(Ly_current, unit)
                Lz_current = np.kron(Lz_current, unit)

        Lx.append(Lx_current)
        Ly.append(Ly_current)
        Lz.append(Lz_current)

    H = np.zeros((2**nspins, 2**nspins), dtype=np.complex128)
    for n in range(nspins):
        H += v[n] * Lz[n]

    for n in range(nspins):
        for k in range(nspins):
            if n != k:
                H += 0.5 * J[n, k] * (Lx[n]*Lx[k] + Ly[n]*Ly[k] + Lz[n]*Lz[k])

    return H

In [9]:
from speedtest.kuprov import kuprov_H_dense

In [14]:
dense_result = %timeit -o kuprov_H_dense(*spin8())
dense_result.average

309 ms ± 35.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


0.309401860286016